## Import necessary module 

In [153]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests

import time

from pandas.io.json import json_normalize
from geojson import Polygon
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox

from sentinelhub import SentinelHubRequest, SentinelHubDownloadClient, DataSource, \
    MimeType, DownloadRequest, CRS, BBox

import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from shapely.geometry import Polygon
import re
import os

from pathlib import Path


## Extract date from SentinelHubImage-available_dates.xlsx	 

In [154]:
df3 = pd.read_excel('https://bitbucket.org/adoval4/space-ag-data-analyst-challenge/raw/37385be5a0c78d3b98b64ccc99e6172d36ad68ea/SentinelHubImage-available_dates.xlsx', index=False)
#print(df3)
date = df3[['_date']]


print(date)

         _date
0   2020-04-20
1   2020-04-15
2   2020-03-11
3   2020-02-15
4   2020-02-10
..         ...
84  2016-12-22
85  2016-12-02
86  2016-11-22
87  2016-09-13
88  2016-07-15

[89 rows x 1 columns]


## Extract Geometry from features

In [155]:
json_data = 'https://bitbucket.org/adoval4/space-ag-data-analyst-challenge/raw/37385be5a0c78d3b98b64ccc99e6172d36ad68ea/farm_map.json'

df1 = gpd.read_file(json_data)
df1 = df1[['geometry']]
polygon = df1
df1 = df1.bounds
print(df1)


         minx       miny       maxx       maxy
0  -75.642984 -14.200629 -75.639658 -14.197876
1  -75.645651 -14.201559 -75.642317 -14.198803
2  -75.642317 -14.202454 -75.638989 -14.199701
3  -75.644984 -14.203384 -75.641650 -14.200629
4  -75.641650 -14.204280 -75.638321 -14.201527
5  -75.644319 -14.205212 -75.640982 -14.202454
6  -75.640982 -14.206095 -75.637657 -14.203352
7  -75.643652 -14.207029 -75.640319 -14.204280
8  -75.640319 -14.207924 -75.636986 -14.205165
9  -75.642990 -14.208854 -75.639650 -14.206095
10 -75.639650 -14.209740 -75.636322 -14.206998
11 -75.642324 -14.210673 -75.638987 -14.207924
12 -75.638987 -14.211560 -75.635656 -14.208810
13 -75.641661 -14.212492 -75.638321 -14.209740
14 -75.638321 -14.213385 -75.634988 -14.210631
15 -75.640998 -14.214319 -75.637654 -14.211560
16 -75.637654 -14.215142 -75.634345 -14.212454
17 -75.640332 -14.216077 -75.637012 -14.213385
18 -75.647639 -14.196068 -75.644346 -14.193356
19 -75.652318 -14.199735 -75.649033 -14.197043
20 -75.643642

## Extract property values and area from features

NVDI is calculated 
$ \frac{NIR - RED}{NIR + RED} $

NIR - reflection in the near-infrared spectrum
RED - reflection in the red range of the spectrum

I was looking into farm_map.json, but there was no such value that can be NVDI (between -1 and 1).
Thus, I decided to use Crop Yield (Ha/t). Following code shows crop yield, and all values from 2019-09-19

In [252]:
import re

df2 = gpd.read_file(json_data)
df2 = df2[['values', 'unit']]
df2 = df2.astype(str)
df2['values'] = df2['values'].apply(lambda x: x.replace('[','').replace(']','').replace('{', '').replace('}', ''))
df2['unit'] = df2['unit'].apply(lambda x: x.replace('[','').replace(']','').replace('{', '').replace('}', ''))

df2['values'] = df2['values'].apply(lambda x: x.replace(x, x.split(',')[0]))
df2['unit'] = df2['unit'].apply(lambda x: x.replace(x, x.split(',')[0]))

df2['values'] = df2['values'].apply(lambda x: re.sub(r'[a-z]+', '', x, re.I))
df2['values'] = df2['values'].apply(lambda x: x.replace('  "": ', ''))

df2['unit'] = df2['unit'].apply(lambda x: re.sub(r'[a-z]+', '', x, re.I))
df2['unit'] = df2['unit'].apply(lambda x: x.replace("state': 'area': ", ''))
df2['unit'] = df2['unit'].apply(lambda x: x.replace("'': '':", ''))

df2 = df2.astype(float)
df2['crop yield'] = df2['values'] / 1000 / df2['unit']
 
print(df2.head())

        values      unit  crop yield
0  3581.158766  6.530601    0.548366
1  4037.375179  6.555069    0.615916
2  4658.420440  6.533254    0.713032
3  4196.272166  6.551013    0.640553
4  4744.580970  6.534646    0.726066


## Plot image 

In [8]:
def plot_image(image, factor=1):
    """
    Utility function for plotting RGB images.
    """
    fig = plt.subplots(nrows=1, ncols=1, figsize=(15, 7))

    if np.issubdtype(image.dtype, np.floating):
        plt.imshow(np.minimum(image * factor, 1))
    else:
        plt.imshow(image) 

## Save the NDVI images of each batch and each date

1. Assign directory value as a global
2. Loop to save image with each polygon location and each date
3. Get the list of file name from the directory
4. Create a DataFrame including date, polygon location, and file name. 

In [150]:
global directory 

directory = '/Users/gbeak/Desktop/Study Materials/img'

for i in range(0, len(df1)):
    print('initiate ', i + 1, 'th row')
    bestiboka_coords_wgs84 = df1.iloc[i].values.tolist()
    bestiboka_bbox = BBox(bbox=bestiboka_coords_wgs84, crs=CRS.WGS84)
    time.sleep(1)
    for j in range(0, len(df3)):
        print('initiate ', j + 1, 'th date')
        for folder, _, filenames in os.walk(directory):
            for filename in filenames:
                os.path.join(folder, filename)
                time.sleep(1)
        date = df3['_date'].iloc[j]
        wms_true_color_request = WmsRequest(
            data_folder = directory,
            layer='TRUE-COLOR-S2-L1C',
            bbox=betsiboka_bbox,
            time=date,
            width=512,
            height=856)
        time.sleep(1)
        wms_true_color_img = wms_true_color_request.get_data(save_data=True)
        print('save image from', i + 1, 'th row and ', j + 1, 'th date')
        time.sleep(1)


initiate  1 th row
initiate  1 th date
save image from 1 th row and  1 th date
initiate  2 th date
save image from 1 th row and  2 th date
initiate  3 th date
save image from 1 th row and  3 th date
initiate  4 th date
save image from 1 th row and  4 th date
initiate  5 th date
save image from 1 th row and  5 th date
initiate  6 th date
save image from 1 th row and  6 th date
initiate  7 th date
save image from 1 th row and  7 th date
initiate  8 th date
save image from 1 th row and  8 th date
initiate  9 th date
save image from 1 th row and  9 th date
initiate  10 th date
save image from 1 th row and  10 th date
initiate  11 th date
save image from 1 th row and  11 th date
initiate  12 th date
save image from 1 th row and  12 th date
initiate  13 th date
save image from 1 th row and  13 th date
initiate  14 th date
save image from 1 th row and  14 th date
initiate  15 th date
save image from 1 th row and  15 th date
initiate  16 th date
save image from 1 th row and  16 th date
initiat

KeyboardInterrupt: 

In [244]:
new_df = pd.DataFrame()
new_df1 = pd.DataFrame()

n = len(polygon)
new_df['Date'] = np.resize(list(date.iloc[0]), n)
new_df['Polygon'] = polygon

for i in range(1, len(date)):
    var1 = np.resize(list(date.iloc[i]), n)
    new_df1['Date'] = var1
    new_df1['Polygon'] = polygon
    new_df = new_df.append(new_df1)
    
print(new_df)


          Date                                            Polygon
0   2020-04-20  POLYGON ((-75.6396578867764 -14.19970133595172...
1   2020-04-20  POLYGON ((-75.64231663022875 -14.2006288241436...
2   2020-04-20  POLYGON ((-75.63898926117189 -14.2015272683264...
3   2020-04-20  POLYGON ((-75.64164955317383 -14.2024540244683...
4   2020-04-20  POLYGON ((-75.64164955317383 -14.2024540244683...
..         ...                                                ...
28  2016-07-15  POLYGON ((-75.64968970670726 -14.1970429501923...
29  2016-07-15  POLYGON ((-75.64430202199843 -14.1951328024683...
30  2016-07-15  POLYGON ((-75.64698264826821 -14.1960684956183...
31  2016-07-15  POLYGON ((-75.64964409685477 -14.1970270320072...
32  2016-07-15  POLYGON ((-75.64898805011781 -14.1988135082976...

[2937 rows x 2 columns]


In [242]:
paths = sorted(Path(directory).iterdir(), key=os.path.getmtime)
paths = list(paths)
new_df2 = pd.DataFrame()
new_df2['file'] = paths
new_df2 = new_df2[:-1]
new_df2['file'] = new_df2['file'].apply(str)
new_df2['file'] = new_df2['file'].apply(lambda x: x.replace("/Users/gbeak/Desktop/Study Materials/img/", ""))


print(new_df2)

                                file
0   9f0155233f1f9260a6bfbcb1f3cdb830
1   cdcb1f4b0b37321293fd68b4656eb565
2   981484182e0ebff1a1adb7a4ea947e45
3   8c06d111b2062571aa7d97b8536bda89
4   6c9098333267d82727e355fbdb6484f0
..                               ...
84  ba66a3b080c4370efa17e04eb07c6c8d
85  fd0cee28789bf90358348060c242ae8c
86  e3cb34d5ead6d0d90d353c23d7063cee
87  206b0a354debe672dd62b1cf2a4608fc
88  49d251a35e4d5181ed61d9afeb3a0045

[89 rows x 1 columns]


In [251]:
new_df = pd.concat([new_df, new_df2])
new_df['file'] = new_df['file'].shift(-83)
new_df = new_df.dropna()
new_df

,Date,Polygon,file
30,2017-01-01,POLYGON ((-75.64698264826821 -14.1960684956183...,9f0155233f1f9260a6bfbcb1f3cdb830
31,2017-01-01,POLYGON ((-75.64964409685477 -14.1970270320072...,cdcb1f4b0b37321293fd68b4656eb565
32,2017-01-01,POLYGON ((-75.64898805011781 -14.1988135082976...,981484182e0ebff1a1adb7a4ea947e45
0,2016-12-22,POLYGON ((-75.6396578867764 -14.19970133595172...,8c06d111b2062571aa7d97b8536bda89
1,2016-12-22,POLYGON ((-75.64231663022875 -14.2006288241436...,6c9098333267d82727e355fbdb6484f0
...,...,...,...
28,2016-07-15,POLYGON ((-75.64968970670726 -14.1970429501923...,e79de4d20f89fa5e05796330a3dc96ea
29,2016-07-15,POLYGON ((-75.64430202199843 -14.1951328024683...,d1f594ae7816c55e743432563b8c5fea
30,2016-07-15,POLYGON ((-75.64698264826821 -14.1960684956183...,a4d8eef27c53d1d0bd29d36851b16fd0
31,2016-07-15,POLYGON ((-75.64964409685477 -14.1970270320072...,0a4c7b0cbf674d27df313200f6e8496b


## If I had more time ... 

- Solve the loop which creates image files because it was stopped.
- Solve the quality of image
- Add image file in file column